In [1]:
import pyscal.crystal_structures as pcs
import pyscal.csystem as pcm
import pyscal.core as pc
from ase import atoms
from ase.io import read, write
from ase.build import bulk
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nx = 1
satoms, box = pcs.make_crystal(structure='bcc', 
    lattice_constant=3.127, 
    repetitions=(nx,nx,nx),)
print(len(satoms["positions"]))
sys = pc.System()

2


In [6]:
satoms['positions'][1] = [-1.5635, -1.5635, -1.5635]

In [7]:
satoms

{'positions': [[0.0, 0.0, 0.0], [-1.5635, -1.5635, -1.5635]],
 'ids': [1, 2],
 'types': [1, 1],
 'ghost': [False, False]}

In [8]:
sys.box = box
sys.atoms = satoms

In [16]:
a = bulk('Mo')
sys = pc.System()
sys.read_inputfile(a, format="ase")

In [18]:
sys.box

array([[-1.575,  1.575,  1.575],
       [ 1.575, -1.575,  1.575],
       [ 1.575,  1.575, -1.575]])

In [22]:
[-1.575, -1.575, -1.575]

[-1.575, -1.575, -1.575]

In [17]:
sys.boxdims

[8.183940065762943, 8.183940065762943, 8.183940065762943]

In [23]:
pcm.remap_atom_into_box([-1.575, -1.575, -1.575], 
    1, 
    sys.rot, 
    sys.rotinv,
    sys.boxdims)

[3.1499999999999995, 3.1499999999999995, 3.1499999999999995]

In [24]:
sys.atoms["positions"]

[array([0., 0., 0.]),
 array([-1.575, -1.575, -1.575]),
 array([ 0.  ,  0.  , -3.15]),
 array([ 1.575,  1.575, -4.725]),
 array([ 0.  , -3.15,  0.  ]),
 array([ 1.575, -1.575, -1.575]),
 array([ 3.15,  0.  , -3.15]),
 array([ 1.575, -4.725,  1.575]),
 array([ 3.15, -3.15,  0.  ]),
 array([ 4.725, -1.575, -1.575]),
 array([-3.15,  0.  ,  0.  ]),
 array([-1.575,  1.575, -1.575]),
 array([ 0.  ,  3.15, -3.15]),
 array([-1.575, -1.575,  1.575]),
 array([ 1.575,  1.575, -1.575]),
 array([ 0.  , -3.15,  3.15]),
 array([ 1.575, -1.575,  1.575]),
 array([3.15, 0.  , 0.  ]),
 array([-4.725,  1.575,  1.575]),
 array([-3.15,  3.15,  0.  ]),
 array([-1.575,  4.725, -1.575]),
 array([-3.15,  0.  ,  3.15]),
 array([-1.575,  1.575,  1.575]),
 array([0.  , 3.15, 0.  ]),
 array([-1.575, -1.575,  4.725]),
 array([0.  , 0.  , 3.15]),
 array([1.575, 1.575, 1.575])]

In [14]:
9.381-7.817500000000001

1.5634999999999994

Try a fully python numpy based algorithm for finding nearest neighbor

In [10]:
def get_distance(x, box, dim=3):
    box_2 = box/2
    xdum = []
    for i in range(dim):
        xd = np.meshgrid(x[i], x[i])[1] - np.meshgrid(x[i], x[i])[0]
        xdum.append(xd)
    xd = np.array(xdum)
    for i in range(dim):
        xd[i] = np.where(xd[i] > box_2[i], xd[i]-box[i], xd[i])
        xd[i] = np.where(xd[i] < -box_2[i], xd[i]+box[i], xd[i])
    return xd

In [12]:
x = sys.atoms['positions']
box = sys.boxdims

In [16]:
x = np.array(x)

In [20]:
y = x.T

In [28]:
%%timeit
xd = get_distance(y, np.array(box))

224 ms ± 857 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
xd.shape

(3, 2000, 2000)

In [23]:
np.where(xd[]

array([[[ 0.    , -1.5635,  0.    , ...,  1.5635,  3.127 ,  1.5635],
        [ 1.5635,  0.    ,  1.5635, ...,  3.127 ,  4.6905,  3.127 ],
        [ 0.    , -1.5635,  0.    , ...,  1.5635,  3.127 ,  1.5635],
        ...,
        [-1.5635, -3.127 , -1.5635, ...,  0.    ,  1.5635,  0.    ],
        [-3.127 , -4.6905, -3.127 , ..., -1.5635,  0.    , -1.5635],
        [-1.5635, -3.127 , -1.5635, ...,  0.    ,  1.5635,  0.    ]],

       [[ 0.    , -1.5635,  0.    , ...,  1.5635,  3.127 ,  1.5635],
        [ 1.5635,  0.    ,  1.5635, ...,  3.127 ,  4.6905,  3.127 ],
        [ 0.    , -1.5635,  0.    , ...,  1.5635,  3.127 ,  1.5635],
        ...,
        [-1.5635, -3.127 , -1.5635, ...,  0.    ,  1.5635,  0.    ],
        [-3.127 , -4.6905, -3.127 , ..., -1.5635,  0.    , -1.5635],
        [-1.5635, -3.127 , -1.5635, ...,  0.    ,  1.5635,  0.    ]],

       [[ 0.    , -1.5635, -3.127 , ...,  4.6905,  3.127 ,  1.5635],
        [ 1.5635,  0.    , -1.5635, ...,  6.254 ,  4.6905,  3.127 ],
    